## Reti Convoluzionali

### Filtro

Concetto di Filtro: vedi appunti

In [ ]:
Link: https://setosa.io/ev/image-kernels/

In [2]:
# vedi anche 3Blue1brown serie video su NN

In [3]:
from keras.layers import Conv2D

In [5]:
Conv2D(
    filters=64,
    kernel_size=3, # solo 3 perché filtro quadrato
    strides = 2, # di quanto shiftare a sx-dx e up-down, il passo/salto
    # lo stride riduce l'output (ex. 2 dimezza le dimensioni dell'output: le sovrapposizioni diventano la metà, rispetto stride=1)
    padding="same",
    activation = "relu" # serve sempre una attivazione (a meno dell'ultimo layer)
)
# quanti filtri, stride, padding

In [ ]:
# una immagine a colori ha 3 conali: 2 dim spaziali e 3 colori (una dimensione)
# quindi il tensore di partenza (a meno di immagini B&W) a 4 dimensioni:

batch_size x 640 x 480 x 3 (canali)
# vedi anche appunti

In [6]:
import tensorflow as tf

In [7]:
conv2d = Conv2D(
    filters=64,
    kernel_size=3,
    strides = 1,
    padding="same",
    activation = "relu" 
)

In [8]:
img_batch = tf.random.normal([32, 28, 28, 3])
# batch di 32 immagini, ognuna 28x28, a colori (il filtro è 3x3x3)

In [9]:
res = conv2d(img_batch)

In [10]:
# la dimensione di res: 32 elementi, ciascuno 28x28, con 64 canali (perché applico 64 filtri a ogni immagine)
res.shape

TensorShape([32, 28, 28, 64])

In [15]:
# parametri (vedi appunti)
[w.size for w in conv2d.get_weights()]

[1728, 64]

In [ ]:
# 1728 + 64 è il numero dei parametri dello strato conv2d
# da dove derivano:
vedi appunti

# il prossimo strato ottiene qualcosa 28x28 con 64 canali o colori
# filtro 3x3 x 64
vedi appunti

### Pooling

Rete Convo = Filtro + Pooling (vedi appunti)
+ Il filtro (layer convoluzionale) vede le features
+ Il pooling riduce le dimensioni dell'immagine, così che i filtri successivi, anche essendo della stessa shape, vedano una porzione più grande dell'immagine

Quale differenza c'è fra i due? (in quello che fanno)
+ [link 1](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~:text=Pooling%20involves%20selecting%20a%20pooling,a%20stride%20of%202%20pixels.)
+ https://towardsdatascience.com/understanding-convolutions-and-pooling-in-neural-networks-a-simple-explanation-885a2d78f211

### LeNet5 implementation
per la rete vedi wikipedia

In [4]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Lambda
from keras import Sequential
import tensorflow as tf

In [ ]:
#### Lambda Layer
Strato con Stato Interno vuoto, state-less (non ha parametri interni, ex. il flatten, fa solo una operazione). 
Un dense layer è state-full.

Lambda(lambda x: tf.reshape(x, shape=[28,28,1]) # può essere messo al posto della strato Reshape (fa la stessa cosa)
# ingloba una funzione e ne fa uno strato

In [5]:
lenet5 = Sequential(
    [
        Reshape(target_shape = (28,28,1), input_shape=(28,28)),
        Conv2D(filters = 6, kernel_size=5, padding="same",activation="relu" ), 
        # cos'è kernel size? la dimensione del filtro: 5x5
        MaxPooling2D(), # di default stride = pool_size
        Conv2D(filters = 16, kernel_size=5, padding="valid",activation="relu" ), # c'è un motivo per i 16 filtri?
        MaxPooling2D(),
        Flatten(), # perché? perché devo passare l'ouput della convo. all'MLP come singolo vettore?
        #          https://towardsdatascience.com/the-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480#:~:text=Flattening%20is%20converting%20the%20data,called%20a%20fully%2Dconnected%20layer.
        # ora MLP
        Dense(120, "relu"),
        Dense(84, "relu"),
        Dense(10) # o nulla o softmax (output della rete, attento ai logits o probabilità e quindi alla funzione di loss)
    ]
)

In [6]:
lenet5.summary() # sproporzione pesi tra parte Convo e Densa

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 400)               0

In [7]:
from keras.utils import plot_model

In [19]:
!pip install graphviz

In [20]:
plot_model(lenet5, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


#### Applica al mnist

In [15]:
from keras.datasets.mnist import load_data
import numpy as np
from keras.losses  import SparseCategoricalCrossentropy
from keras.optimizers import Adam

In [10]:
(x_train, y_train), (x_test, y_test) = load_data()

In [13]:
x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255

y_train = y_train.astype(np.float32) # metti int32
y_test = y_test.astype(np.float32)

In [16]:
lenet5.compile(
    optimizer=Adam(), 
    loss = SparseCategoricalCrossentropy(from_logits = True), 
    metrics=["Accuracy"])

In [17]:
lenet5.fit(x = x_train, y= y_train, epochs = 20, validation_split=0.2)

Epoch 1/20
1500/1500 [==============================] - 26s 16ms/step - loss: 0.2117 - Accuracy: 0.9360 - val_loss: 0.0771 - val_Accuracy: 0.9766
Epoch 2/20
1500/1500 [==============================] - 23s 15ms/step - loss: 0.0672 - Accuracy: 0.9795 - val_loss: 0.0617 - val_Accuracy: 0.9816
Epoch 3/20
1500/1500 [==============================] - 25s 16ms/step - loss: 0.0480 - Accuracy: 0.9847 - val_loss: 0.0580 - val_Accuracy: 0.9827
Epoch 4/20
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0377 - Accuracy: 0.9881 - val_loss: 0.0476 - val_Accuracy: 0.9854
Epoch 5/20
1500/1500 [==============================] - 23s 16ms/step - loss: 0.0302 - Accuracy: 0.9905 - val_loss: 0.0522 - val_Accuracy: 0.9844
Epoch 6/20
1500/1500 [==============================] - 23s 15ms/step - loss: 0.0247 - Accuracy: 0.9919 - val_loss: 0.0508 - val_Accuracy: 0.9853
Epoch 7/20
1500/1500 [==============================] - 23s 15ms/step - loss: 0.0220 - Accuracy: 0.9929 - val_loss: 0.0467 -

#### Commento
+ MLP finale: nelle archit. moderne non c'è o è meno gigante
+ Reti Convo anche per altri input (ex. elettrocardiogrammi...) (con dimensioni diverse)
+ https://www.image-net.org/ dataset imm.

## Label Studio
+ [link](https://labelstud.io/)
+ labeling dei dati (di tutti i tipi)
+ anche in team

In [ ]:
# Segmentazione Semantica: taggare pixel per pixel (ex. in ambito medico, tessuti ...)

In [ ]:
# run label-studio: try it out
# è possibile un pre-labeling con modelli di ML

In [ ]:
# usalo in un conda env

## BackPropagation

In [ ]:
vedi appunti